In [81]:
### Create interactive map of residents living within 400 - 1600 m (+ 250 m because of the grid resolution) walking distance from 
### metro and train station in Finland's capital region with a value slider. Data for the map is loaded from open data services of 
### Maanmittauslaitos (MML) and Helsinki Region Environmental Services Authority (HSY).

### IMPORT DATA ###

# Import modules
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopandas.tools import geocode
import numpy as np
from pyproj import CRS
import requests
import geojson
import matplotlib.pyplot as plt
from shapely.ops import cascaded_union
import mapclassify
import contextily as ctx
from mpl_toolkits.axes_grid1 import make_axes_locatable

import folium
import folium.plugins
import branca
import branca.colormap as cm

## Read shape file containing the capital region as polygons into variable 'grid' (Data from https://tiedostopalvelu.maanmittauslaitos.fi/tp/kartta)
# File path
fp_grid = "data/pkseutu.shp"

# Read in data
grid = gpd.read_file(fp_grid)

# Check if crs is correct and set crs to ETRS89 / TM35FIN if the crs is not defined correctly
if (grid.crs != "epsg:3067"):    
    grid = grid.set_crs(epsg=3067)
# Reproject to WGS 84 / Pseudo-Mercator if the crs is not defined correctly
if (grid.crs != "epsg:4326"):    
    grid = grid.to_crs(epsg=4326)

# Combine polygons of each city to form one polygon of the whole capial region
grid['constant'] = 0
boundary = grid.dissolve(by='constant')

# Check the data
print(grid.head())
print(grid.crs)
print(boundary)

     GML_ID NATCODE     NAMEFIN      NAMESWE  LANDAREA  FRESHWAREA  SEAWAREA  \
0  27817426     091    Helsinki  Helsingfors    214.25        0.91    500.32   
1  27806267     049       Espoo         Esbo    312.32       17.91    197.80   
2  27817421     092      Vantaa        Vanda    238.37        1.98      0.00   
3  27806296     235  Kauniainen    Grankulla      5.89        0.11      0.00   

   TOTALAREA                                           geometry  constant  
0     715.48  POLYGON ((25.19025 60.28568, 25.19027 60.28567...         0  
1     528.03  POLYGON ((24.83140 60.25406, 24.83168 60.25321...         0  
2     240.35  POLYGON ((25.19025 60.28568, 25.18958 60.28410...         0  
3       6.00  POLYGON ((24.73904 60.21885, 24.73923 60.21905...         0  
epsg:4326
                                                   geometry    GML_ID NATCODE  \
constant                                                                        
0         POLYGON ((25.19025 60.28568, 25.19027

In [82]:
## Read population grid data for 2018 into a variable `pop`. 

# Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format).
params = dict(service='WFS',
              version='2.0.0',
              request='GetFeature',
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018',
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

# Clean out unnecessary columns
pop = pop[["asukkaita", "geometry"]]

# Set crs to ETRS89 / GK25FIN and reproject to WGS 84 / Pseudo-Mercator if the crs is not defined correctly
if (pop.crs == None):    
    pop = pop.set_crs(epsg=3879)
if (pop.crs != "epsg:4326"):    
    pop = pop.to_crs(epsg=4326)

# Check the data
print(pop.head())
print(pop.crs)

   asukkaita                                           geometry
0        108  MULTIPOLYGON Z (((24.57654 60.18042 0.00000, 2...
1        273  MULTIPOLYGON Z (((24.58102 60.18267 0.00000, 2...
2        239  MULTIPOLYGON Z (((24.58538 60.19391 0.00000, 2...
3        202  MULTIPOLYGON Z (((24.58541 60.19166 0.00000, 2...
4        261  MULTIPOLYGON Z (((24.58544 60.18942 0.00000, 2...
epsg:4326


In [83]:
## Read buffer polygons that describe 400 m, 800 m, 1200 m and 1600 m accessibilities via pedestrian and bicycle ways from metro and 
## train stations 

# Save wanted buffer sizes in a list which is used in loading the data
dists = ['400', '800', '1200', '1600']

# Create an empty geopandas GeoDataFrame for the data
buffs = gpd.GeoDataFrame()

# Iterate through wanted buffer distance list
for dist in dists:

    # Specify the url for web feature service and typeName of the data layer
    url_buff = 'https://kartta.hsy.fi/geoserver/wfs'
    type_name = dist + 'm_verkostobufferi'

    # Specify parameters (read data in json format).
    params_buff = dict(service='WFS',
                  version='2.0.0',
                  request='GetFeature',
                  typeName=type_name,
                  outputFormat='json')

    # Fetch data from WFS using requests
    r = requests.get(url_buff, params=params_buff)

    # Create GeoDataFrame from geojson
    buff = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

    # Clean out unnecessary columns
    buff = buff[["asema", "geometry"]]

    # Set crs to ETRS89 / GK25FIN and reproject to WGS 84 / Pseudo-Mercator if the crs is not defined correctly
    if (buff.crs == None):    
        buff = buff.set_crs(epsg=3879)
    if (buff.crs != "epsg:4326"):    
        buff = buff.to_crs(epsg=4326)

    # Clip out stations that are located outside the capital region
    clip_mask = buff.within(boundary.at[0,'geometry'])
    buff = buff.loc[clip_mask]
    
    # Create column which indicates buffer distance for the slider
    buff['dist'] = dist

    # Check the data
    print(buff.head(1))
    print(len(buff))

    # Add the data to combined GeoDataFrame
    buffs = buffs.append(buff)
    
# Check output of the loop
buffs.head()

    asema                                           geometry dist
0  Käpylä  MULTIPOLYGON (((24.94901 60.22312, 24.94936 60...  400
64
  asema                                           geometry dist
0  Kilo  MULTIPOLYGON (((24.78703 60.21349, 24.78686 60...  800
64
       asema                                           geometry  dist
1  Myllypuro  MULTIPOLYGON (((25.07919 60.23449, 25.07894 60...  1200
64
  asema                                           geometry  dist
0  Kera  MULTIPOLYGON (((24.73268 60.21786, 24.73320 60...  1600
63


,asema,geometry,dist
0,Käpylä,"MULTIPOLYGON (((24.94901 60.22312, 24.94936 60...",400
1,Lentoasema,"MULTIPOLYGON (((24.96643 60.31256, 24.96648 60...",400
2,Hiekkaharju,"MULTIPOLYGON (((25.05169 60.30546, 25.05171 60...",400
4,Helsinki,"MULTIPOLYGON (((24.94144 60.17622, 24.94145 60...",400
6,Kannelmäki,"MULTIPOLYGON (((24.87353 60.24244, 24.87387 60...",400


In [84]:
### PROCESS DATA ###

# Create new column to 'buffs' where total resident amounts within each buffer areas are stored 
buffs["residents_sum"] = None

# Create a spatial join between grid layer and buffer layer. "Intersects" option used here to include all grid cells which 
# touch the buffer area (NOTE that with this choice the accuracy of the buffers is lost due to the grid resolution)
pop_combined = gpd.sjoin(pop, buffs, how="left", op="intersects")

# Group the data by both train and metro station names AND distance classes
groupedA = pop_combined.groupby(['asema','dist'])

groupedA.head()

,asukkaita,geometry,index_right,asema,dist,residents_sum
0,108,"MULTIPOLYGON Z (((24.57654 60.18042 0.00000, 2...",49.0,Mankki,400,None
0,108,"MULTIPOLYGON Z (((24.57654 60.18042 0.00000, 2...",4.0,Mankki,800,None
0,108,"MULTIPOLYGON Z (((24.57654 60.18042 0.00000, 2...",72.0,Mankki,1200,None
1,273,"MULTIPOLYGON Z (((24.58102 60.18267 0.00000, 2...",49.0,Mankki,400,None
1,273,"MULTIPOLYGON Z (((24.58102 60.18267 0.00000, 2...",4.0,Mankki,800,None
...,...,...,...,...,...,...
3119,866,"MULTIPOLYGON Z (((25.13527 60.21019 0.00000, 2...",21.0,Vuosaari,400,None
3120,720,"MULTIPOLYGON Z (((25.13526 60.20795 0.00000, 2...",21.0,Vuosaari,400,None
3121,1075,"MULTIPOLYGON Z (((25.13525 60.20571 0.00000, 2...",21.0,Vuosaari,400,None
3122,1164,"MULTIPOLYGON Z (((25.13524 60.20346 0.00000, 2...",21.0,Vuosaari,400,None


In [85]:
# Store sum of residents living approximately 400 m, 800 m, 1200 m and 1600 m from station to column "sum" 
# (the distance doesn't stay constant in performed analysis but accurate enough for this visualization)
for name, group in groupedA:
    buffs.loc[(buffs["asema"]==name[0]) & (buffs['dist']==name[1]),'residents_sum'] = group["asukkaita"].agg("sum")
    
    
## Convert the buffer polygons to points (location set as centroids of 400 m buffers, approximate of the station locations)
point_data = buffs
point_data = point_data.reset_index()

# Replace NoData in residents_sum column with 0
point_data["residents_sum"] = point_data["residents_sum"].replace(to_replace=np.nan, value=0)

# Group the data by only train and metro station names
groupedB = point_data.groupby('asema')

# Convert to points based on centroids
for name, group in groupedB:
    point_data.loc[point_data["asema"]==name,'geometry'] = group['geometry'].centroid

point_data.head()

<ipython-input-85-992fe491984d>:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  point_data.loc[point_data["asema"]==name,'geometry'] = group['geometry'].centroid


,index,asema,geometry,dist,residents_sum
0,0,Käpylä,POINT (24.94670 60.22007),400,1252
1,1,Lentoasema,POINT (24.96789 60.31475),400,0
2,2,Hiekkaharju,POINT (25.04942 60.30370),400,1920
3,4,Helsinki,POINT (24.94114 60.17210),400,885
4,6,Kannelmäki,POINT (24.87614 60.23966),400,7698


In [86]:
# Assign same point (centroid of 400 m buffer polygon) for each data row of same stations
### THIS PART NOT DONE YET ###

# Reorganize the column order
point_data = point_data[["geometry","asema","residents_sum", "dist"]]

# Create a Natural Breaks classifier
classifier = mapclassify.NaturalBreaks.make(k=14)

# Classify the data and add 1 to class values to avoid zeros
point_data["class"] = point_data[['residents_sum']].apply(classifier)
point_data["class"] = point_data['class'] + 1

#point_data['x'] = point_data['geometry'].apply(lambda p: p.x)
#point_data['y'] = point_data['geometry'].apply(lambda p: p.y)
    
#point_data['lon'] = point_data.geometry.apply(lambda p: int(p.x))
#point_data['lat'] = point_data.geometry.apply(lambda p: int(p.y))

# Get x and y coordinates for each point
point_data["x"] = point_data["geometry"].apply(lambda geom: geom.x)
point_data["y"] = point_data["geometry"].apply(lambda geom: geom.y)

# Create a list of coordinate pairs
locations = list(zip(point_data["y"], point_data["x"]))

# Check data    
print(point_data.tail())

                      geometry          asema  residents_sum  dist  class  \
250  POINT (24.86242 60.24831)  Malminkartano          27725  1600      9   
251  POINT (24.68015 60.20380)      Tuomarila          20512  1600      7   
252  POINT (25.02790 60.26336)       Tapanila          21699  1600      8   
253  POINT (24.60040 60.18893)      Kauklahti           7368  1600      3   
254  POINT (24.92039 60.16431)     Ruoholahti          48880  1600     12   

             x          y  
250  24.862423  60.248313  
251  24.680154  60.203797  
252  25.027898  60.263355  
253  24.600399  60.188933  
254  24.920391  60.164314  


In [87]:
# Divide data from each buffer distances into separate GeoDataFrames
buff400 = point_data.loc[point_data['dist']=='400']
buff800 = point_data.loc[point_data['dist']=='800']
buff1200 = point_data.loc[point_data['dist']=='1200']
buff1600 = point_data.loc[point_data['dist']=='1600']

In [96]:
### PLOT DATA

m = folium.Map(location=[60.20426, 24.96179], tiles = 'cartodbpositron',
                zoom_start=11, control_scale=True, prefer_canvas=True)

# Create colormap
colormap = cm.LinearColormap(colors=['yellow','red'], index=[0,85000],vmin=0,vmax=max(point_data['residents_sum']))

def station_style(station):
    def station_colors(value):
        station_colors = {'400':'orange','800':'blue','1200':'red','1600':'silver'}
        # Create colormap
        station_colors = cm.LinearColormap(colors=['yellow','red'], index=[0,85000],vmin=0,vmax=85000)
        return station_colors.get(value) #return station_colors.get(line,'grey')
    return folium.CircleMarker(
        radius=3,
        location=(station.y, station.x), #location=(station.geometry.y, station.geometry.x),
        color=station_colors(station.dist),
        tooltip=station.asema + ", \n" + str(station.residents_sum) + " residents",
        fill=True,
        fill_opacity=.8)

stations400 = folium.FeatureGroup(name='Residents within 400 m from metro and train stations')
stations800 = folium.FeatureGroup(name='Residents within 800 m from metro and train stations')
stations1200 = folium.FeatureGroup(name='Residents within 1200 m from metro and train stations')
stations1600 = folium.FeatureGroup(name='Residents within 1600 m from metro and train stations')

for station in buff400.itertuples():
    station_style(station).add_to(stations400)
    
for station in buff800.itertuples():
    station_style(station).add_to(stations800)
    
for station in buff1200.itertuples():
    station_style(station).add_to(stations1200)
    
for station in buff1600.itertuples():
    station_style(station).add_to(stations1600)

stations400.add_to(m)
stations800.add_to(m)
stations1200.add_to(m)
stations1600.add_to(m)

# Create and add a layer control object 
folium.LayerControl().add_to(m)

# Add legend 
m.addLayer(colormap)

m
#outfp = "choropleth_map.html"
#m.save(outfp)